In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

# Baseball
## Motion of Batted and Pitched Balls

## Lecture 5

Let's move on from cannon shells to a more friendly game of baseball.  If we want to understand the dynamics of a ball that is hit by a batter or thrown by a pitcher, the approach is still use projectile motion like we did in the previous lecture.

The air drag turns out to be again critically important for correctly predicting the trajectory and range of baseball.  Air drag depends on speed in two ways.  The first is directly, as in

$$ F_{drag} = \frac{1}{2} \rho C A v^2 $$

where the drag force is proportional to the square of the velocity (as well as the cross-sectional area $A$ and air density $\rho$.  

### Velocity dependent drag

The second is that the coefficient $C$ itself is also a function of $v$.

![Drag Coefficient for Baseball](http://farside.ph.utexas.edu/teaching/329/lectures/img362.png)

This plot is taken from a book called *The Physics of Baseball* and appears to be based on experimental data.  The drop in drag coefficient is due to a transition from a laminar flow around the ball to a turbulent flow around the ball.


The textbook by Giordano offers the following approximation for the drag of of a normal baseball (in SI units):

$$ \frac{B_2}{m} = 0.0039 + \frac{0.0058}{1+\exp[(v-v_d)/\Delta]}$$

where $v_d$ = 35 m/s and $\Delta$ = 5 m/s. We call this kind of approximate formula a *parametrization*.  Note that the details of factors such as air density, mass, cross-sectional area and so on are all contained with in this value $B_2/m$.

To use it a numerical model, we can define a new function that calculates this formula for us with `v` being passed in as an argument.

In [ ]:
def Bm(v):
    vd = 35 # m/s
    Δ = 5 # m/s
    Bm = 0.0039 + 0.0058 / (1 + np.exp(v-vd) / Δ)
    return Bm

Python code can be written using Unicode (not just ASCII letters and numbers).  The capital delta, $\Delta$, could also be replaced with the word `Delta` (and it often is in other programming languages).

To type this character (assuming you do not have a Greek keyboard) Jupyter has a neat feature that if you type
    
`\Delta<TAB>`

in a code cell, it is automatically turned into a Δ symbol. 

A wonderful property of using numpy functions like `np.exp` is that it can handle both individual numbers,

In [ ]:
print(Bm(30))

or evaluate an entire array in with one function call

In [ ]:
v = np.arange(20, 60, 10)
print(Bm(v))

Let's decrease the stepsize for `v` and plot $B_2/m$ vs $v$.

In [ ]:
v = np.arange(20, 60, 1)
fig, axes = plt.subplots()
plt.plot(v, Bm(v))
plt.xlabel("$v$ (m/s)")
plt.ylabel("$B_2 / m$ (m$^{-1}$)")
plt.show()

Going forward instead of using a constant variable called `Bm` that will represent the $B_2/m$ factor, we will calculate `Bm(v)` as a function of the velocity as needed.

### Wind and Relative Velocity

Another variation we can include is to account for the effects of a tail or head wind. Let's assume that the wind is only blowing horizontally and has a constant magnitude and direction during the flight of the ball.

The components of the drag force become


\begin{align}
F_{drag,x} &= - B_2 \left| \vec{v} - \vec{v}_{wind} \right| (v_x - v_{wind} )\\
F_{drag,y} &= - B_2 \left| \vec{v} - \vec{v}_{wind} \right| v_y
\end{align}

This because air drag is due the *relative velocity* of the ball with repect to the air.


### Numerical Solution

Now let's use Euler's method to determine the path of projectile with an initial velocity.  While now we are modeling baseballs instead of cannon shells, the numerical code is basically the same as it was from last lecture with the addition of the new `Bm(v)` function and the effect of the wind.

#### Initialization

In [ ]:
# acceleration due to gravity, m/s^2
g = 9.81

# time step, s
dt = 0.01

# max time, s
tmax = 200

#### Computation

In [ ]:
def update(i, x, y, vx, vy, v_wind=0):
    
    # update positions
    x[i+1] = x[i] + vx*dt
    y[i+1] = y[i] + vy*dt
        
    # calculate relative velocity magnitude
    v = np.sqrt((vx - v_wind)**2 + vy**2)
        
    # compute air drag
    Fdrag_x = -(Bm(v))*v*(vx-v_wind)
    Fdrag_y = -(Bm(v))*v*vy
        
    # update velocities
    vx_new = vx + Fdrag_x * dt
    vy_new = vy + (-g + Fdrag_y) * dt
    
    return vx_new, vy_new

In [ ]:
def solve(theta=45, v0=70, y0=0, v_wind=0):
    """
    Given 
        theta, initial angle (degrees)
        v0, initial speed (m/s)
        y0, initial height above the ground (m)
        v_wind, wind (horizontal) velocity (m/s)
    Returns
        x,y trajectory of a projectile
    """
    
    # Allocate memory for arrays and set to zero
    N = round(tmax / dt)
    x = np.zeros(N)
    y = np.zeros(N)
    
    # Initial values
    x[0] = 0
    y[0] = y0
    vx = v0*np.cos(np.deg2rad(theta))
    vy = v0*np.sin(np.deg2rad(theta))

    # Calculate the solution
    for i in range(N-1):
        vx, vy = update(i, x, y, vx, vy, v_wind)
        
        # stop when y <= 0
        if y[i+1] <= 0:
            break

    # truncate arrays to when projectile has hit the ground (y<=0)
    N = i+2
    x = x[:N]
    y = y[:N]

    # interpolate final position if projectile is below ground level
    if y[i+1] < 0:
        r = -y[i]/y[i+1]
        x[i+1]=(x[i]+r*x[i+1])/(r+1)
        y[i+1]=0 

    return x, y

#### Plotting

In [ ]:
def plot(x, y, label='', style='-'):
    plt.title('Trajectory of a baseball')

    plt.plot(x, y, style, label=label)
    plt.xlabel('x (m)')
    plt.ylabel('y (m)')

#### Driver

In [ ]:
theta = 35 # degrees
v0 = 50 # m/s (about 110 mph)
y0 = 2 # m

fig, axes = plt.subplots()
for v_wind in [-5, 0, 5]:
    x, y = solve(theta=theta, 
                 v0=v0, y0=y0, 
                 v_wind=v_wind)
    plot(x,y, label='v_wind = {}'.format(v_wind)) 
    
plt.legend()
plt.show()

### Exercise
> 1. Go back and review the code in detail. Any questions?
> 2. Try using `fig, axes = plt.subplots(figsize=(10,4))` and see what happens
> 3. Add the line `plt.axis('equal')` to the driver code just before the `plt.show()` function and rerun.

## Effects of spin

We've seen air drag can considerable change the range of a projectile - for both cannon shells and baseballs. In addition, it turns out that *spin* can radically affect the trajectory of projectile. This is called the Magnus effect or Magnus Force.



### Magnus Force

* [What Is The Magnus Force?](https://www.youtube.com/watch?v=23f1jvGUWJs)

Let's model this so-called Magnus force and add it in to our computational model for projectile motion.  Consider the drag acting on either side of a spinning ball rotating at a angular velocity $\omega$.

See figure 2.8 in Giordano

We know that $F_{drag}$ depends on the relative velocity of the ball with the air.  When the ball is spinning, one side is moving faster and one side is moving slower relative to the air. Hence, the drag forces on each side are not the same.

The details of calculating the Magnus force directly is beyond the scope of this course (requires Fluid Dynamics) but we can understand the functional form of the Magnus force.

The drag force is proportional to the square of the relative velocity. On the one side this is  $(v + r \omega))^2$ while on the other it is $(v - r\omega$)^2.  The Magnus force is proportional to the difference between these two terms:

$$ F_m \propto (v+r\omega)^2 - (v-r\omega)^2 \sim v r \omega\;.$$

So the next spin-dependent force should have a functional form like

$$ F_M = S_0 \omega v_x $$

For a pitched baseball, we can assume that the velocity is predominantly in the $x$-direction. so the Magnus force will act in direction perpendicular to both the $x$-direction and the axis of rotation.   

In the fully general case, we would need to consider a cross-product like 

$$\vec{F}_M = S_0 \vec{\omega} \times \vec{v}$$

for the Magnus effect.

The coefficient $S_0$ is an experimentally determined parameter.  For a $m=149$ g baseball, *Adair (1990)* quotes a value of $S_0 / m \approx 4.1 \times 10^{-4}$. 

To add the effect of the Magnus force into our numerical solution to investigate the dynamics of a curve ball we need to think about our trajectory in all three dimensions.  The equations of motion are


\begin{align}
\frac{d x}{dt} &= v_x \\
\frac{d v_x}{dt} &= - \frac{B_2}{m} v v_x \\
\frac{d y}{dt} &= v_y \\
\frac{d v_y}{dt} &= -g \\ 
\frac{d z}{dt} &= v_z \\
\frac{d v_z}{dt} &= - \frac{S_0}{m} v_x \omega
\end{align}

Since the velocity of a pitched ball is primarily in the $x$-direction, the drag is applied in that direction.   We are assuming here that the drag forces in the $y$- and $z-$ direction are not significant. The axis of rotation is parallel to the $y$-axis (perpendicular to the ground). This is how the textbook sets up the equations.  We could always include more forces if we wanted.



## Code with rotation

Using Euler's method once again gives us the following modified code. Here, I have included the drag forces and the Magnus forces in all three directions for completeness.

In [ ]:
def update(i, 
           rx, ry, rz, 
           vx, vy, vz, 
           ωx, ωy, ωz):
    """
    Update (in-place) the position arrays at time step i+1
    
    rx, ry, rz are arrays given the position r at all time steps
    vx, vy, vz are the velocity components from the i'th time step
    ωx, ωy, ωz are components of the angular velocity vector
    
    Returns the velocity components for time step i+1
    """

    # update positions
    rx[i+1] = rx[i] + vx*dt
    ry[i+1] = ry[i] + vy*dt
    rz[i+1] = rz[i] + vz*dt
        
    # calculate relative velocity magnitude
    v_mag = np.sqrt(vx**2 + vy**2 + vz**2)
        
    # compute air drag
    Fdrag_x = -(Bm(v_mag))*v_mag*vx*0
    Fdrag_y = -(Bm(v_mag))*v_mag*vy*0
    Fdrag_z = -(Bm(v_mag))*v_mag*vz*0
    
    # Magnus force parameter
    Sm = 4.1e-4
    
    # compute Magnus force Fmagnus = S0/m ω x v
    Fmagnus_x = Sm * (ωy*vz - ωz*vy)
    Fmagnus_y = Sm * (ωz*vx - ωx*vz)
    Fmagnus_z = Sm * (ωx*vy - ωy*vx)
        
    # update velocities
    vx_new = vx + (     Fdrag_x + Fmagnus_x) * dt
    vy_new = vy + (-g + Fdrag_y + Fmagnus_y) * dt
    vz_new = vz + (     Fdrag_z + Fmagnus_z) * dt
    
    return vx_new, vy_new, vz_new

In [ ]:
def solve(theta=45, v0=70, y0=0, xmax = 15, ωy=0):
    """
    Given 
        theta, initial angle (degrees)
        v0, initial speed (m/s)
        y0, initial height above the ground (m)
        ωy, angular velocity (rotation axis vertically aligned)
        xmax, distance from pitcher to home plate
        
    Returns
        x,y,z trajectory of a projectile
    """
    
    # Allocate memory for arrays and set to zero
    N = round(tmax / dt)
    x = np.zeros(N)
    y = np.zeros(N)
    z = np.zeros(N)
    
    # Initial values
    x[0] = 0
    y[0] = y0
    z[0] = 0
    vx = v0*np.cos(np.deg2rad(theta))
    vy = v0*np.sin(np.deg2rad(theta))
    vz = 0

    # Calculate the solution
    for i in range(N-1):
        vx, vy, vz = update(i, 
                            x, y, z,
                            vx, vy, vz,
                            0, ωy, 0)
        
        # stop when the ball crosses home plate
        if x[i+1] >= xmax:
            break

    # truncate arrays to when projectile has stopped
    N = i+2
    x = x[:N]
    y = y[:N]
    z = z[:N]
    
    return x, y, z

In [ ]:
theta = 0 # degrees
v0 = 50 # m/s (about 110 mph)
y0 = 1 # m
ωy = 180 # rad/s (about 30 rev/s)
xmax = 30 # m

fig, axes = plt.subplots(figsize=(6,6))

x, y, z = solve(theta=theta, v0=v0, y0=y0, ωy=ωy,
                xmax=xmax)

plot(x, y, label='vertical position (y)')
plot(x, z, label='horizontal position (z)')
plt.xlim(-5, 35)
plt.ylim(-1.2, 1.2)
plt.legend(loc='lower left')
plt.ylabel('y or z (m)')
plt.axvline(0, linestyle=':', color='k')
plt.axvline(xmax, linestyle=':', color='k')
plt.text(0, 0, 'pitcher', rotation=90, 
         horizontalalignment='right',
         verticalalignment='center')
plt.text(xmax, 0, 'home plate', rotation=90, 
         horizontalalignment='left',
         verticalalignment='center')
plt.show()

YouTube video:

* [Surprising Applications of the Magnus Effect](https://www.youtube.com/watch?v=2OSrvzNW9FE )

    

    